In [1]:
%load_ext autoreload
%autoreload 2
import os 
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.85'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'



In [2]:
from nn_ansatz import *
from jax.experimental.optimizers import adam
from jax import tree_util
from tqdm.notebook import trange
from jax import pmap, vmap, grad


In [3]:
# using routines
orbitals = ['isotropic',]
scalar_inputs = [False, ]
n_ps = [3,]

for orbital in orbitals:
    for scalar_input in scalar_inputs:
        for n_p in n_ps:
            if scalar_input:
                if n_p == 3:
                    continue
            name = '%s_si%s_np%i' % (orbital, str(scalar_input), n_p)
            cfg = config = setup(system='LiSolidBCC',
                        n_pre_it=0,
                        n_walkers=512,
                        n_layers=3,
                        n_sh=64,
                        step_size=0.02,
                        n_ph=16,
                        scalar_inputs=scalar_input,
                        orbital_decay=orbital,
                        n_periodic_input=n_p,
                        opt='adam',
                        n_det=4,
                        print_every=100,
                        save_every=2500,
                        lr=1e-4,
                        n_it=7500,
                        name=name)
            log = run_vmc(config)


version 		 140921
seed 		 369
n_devices 		 1
save_every 		 2500
print_every 		 100
exp_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/140921/isotropic_siFalse_np3/adam_1lr-4_1d-3_1nc-4_m512_s64_p16_l3_det4/run1
events_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/140921/isotropic_siFalse_np3/adam_1lr-4_1d-3_1nc-4_m512_s64_p16_l3_det4/run1/events
models_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/140921/isotropic_siFalse_np3/adam_1lr-4_1d-3_1nc-4_m512_s64_p16_l3_det4/run1/models
opt_state_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/140921/isotropic_siFalse_np3/adam_1lr-4_1d-3_1nc-4_m512_s64_p16_l3_det4/run1/models/opt_state
pre_path 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/pretrained/s64_p16_l3_det4_1lr-4_i0.pk
timing_dir 		 /home/amawi/projects/nn_ansatz/src/experiments/LiSolidBCC/140921/isotropic_siFalse_np3/adam_1lr-4_1d-3_1nc-4_m512_s64_p16_l3_det4/run1/events/timing
system 		 LiSolidB

training:   0%|          | 1/7501 [00:00<?, ?it/s]

step 100 | e_mean -5.8512 | e_std 3.0086 | e_mean_mean -5.9659 | acceptance 0.6062 | t_per_it 0.0824 |
step 200 | e_mean -6.3863 | e_std 3.0354 | e_mean_mean -6.4788 | acceptance 0.4973 | t_per_it 0.0849 |
step 300 | e_mean -6.8136 | e_std 3.2933 | e_mean_mean -6.7388 | acceptance 0.4947 | t_per_it 0.0856 |
step 400 | e_mean -6.7450 | e_std 2.5611 | e_mean_mean -6.8211 | acceptance 0.5002 | t_per_it 0.0865 |
step 500 | e_mean -6.8431 | e_std 2.9103 | e_mean_mean -6.9082 | acceptance 0.5057 | t_per_it 0.0871 |
step 600 | e_mean -6.9718 | e_std 2.9579 | e_mean_mean -7.0146 | acceptance 0.4980 | t_per_it 0.0873 |
step 700 | e_mean -7.1974 | e_std 3.1755 | e_mean_mean -7.0631 | acceptance 0.4961 | t_per_it 0.0874 |
step 800 | e_mean -7.0576 | e_std 2.6690 | e_mean_mean -7.2013 | acceptance 0.4928 | t_per_it 0.0875 |
step 900 | e_mean -7.1736 | e_std 2.4571 | e_mean_mean -7.2298 | acceptance 0.5008 | t_per_it 0.0876 |
step 1000 | e_mean -7.2511 | e_std 5.0622 | e_mean_mean -7.3555 | accepta

In [4]:
log = run_vmc(config)
# save_pk(log.params, 'params.pk')

System: 
 n_atoms = 2 
 n_up    = 3 
 n_down  = 3 
 n_el    = 6 

Ansatz: 
 n_layers = 3 
 n_det    = 4 
 n_sh     = 64 
 n_ph     = 16 

Cell: 
 real_basis: 
 [[6.63 0.   0.  ]
 [0.   6.63 0.  ]
 [0.   0.   6.63]] 
 reciprocal_basis: 
 [[0.94769007 0.         0.        ]
 [0.         0.94769007 0.        ]
 [0.         0.         0.94769007]] 
 real_cut         = 6.00 
 reciprocal_cut   = 6 
 kappa            = 0.50 
 volume           = 291.43 
 min_cell_width   = 6.63 
 n_periodic_input = 1 
 unit_cell_length = 6.63 

converged SCF energy = -12.1628037018806
creating wf
sampling no infs, this could take a while
end sampling no infs


training:   0%|          | 1/7501 [00:00<?, ?it/s]

step 100 | e_mean -9.1831 | e_std 3.3782 | e_mean_mean -9.1150 | acceptance 0.5057 | t_per_it 0.0684 |
step 200 | e_mean -9.6069 | e_std 3.0851 | e_mean_mean -9.4766 | acceptance 0.4895 | t_per_it 0.0685 |
step 300 | e_mean -9.8351 | e_std 2.9581 | e_mean_mean -9.9441 | acceptance 0.5074 | t_per_it 0.0689 |
step 400 | e_mean -10.2764 | e_std 3.1788 | e_mean_mean -10.1491 | acceptance 0.5111 | t_per_it 0.0688 |
step 500 | e_mean -10.8160 | e_std 3.1381 | e_mean_mean -10.5383 | acceptance 0.5025 | t_per_it 0.0687 |


KeyboardInterrupt: 

In [23]:
walkers = None
logger = Logging(**cfg)

keys = rnd.PRNGKey(cfg['seed'])
if bool(os.environ.get('DISTRIBUTE')) is True:
    keys = rnd.split(keys, cfg['n_devices']).reshape(cfg['n_devices'], 2)

mol = SystemAnsatz(**cfg)

vwf = create_wf(mol)
params = initialise_params(mol, keys)
pwf = pmap(create_wf(mol), in_axes=(None, 0))
pwf_grad = grad(lambda x, y: pwf(x, y).sum(), argnums=(0,1))

sampler = create_sampler(mol, vwf)

if cfg['load_pretrain']:
    params, walkers = load_pk(cfg['pre_path'])
elif cfg['pretrain']:
    params, walkers = pretrain_wf(mol, **cfg)

if walkers is None:
    walkers = initialise_walkers(mol, vwf, sampler, params, keys, walkers=walkers)

grad_fn = create_grad_function(mol, vwf)

if cfg['opt'] == 'kfac':
    update, get_params, kfac_update, state = kfac(mol, params, walkers, cfg['lr'], cfg['damping'], cfg['norm_constraint'])
elif cfg['opt'] == 'adam':
    init, update, get_params = adam(cfg['lr'])
    update = jit(update)
    state = init(params)
else:
    exit('Optimiser not available')

steps = trange(1, cfg['n_it']+1, initial=1, total=cfg['n_it']+1, desc='training', disable=None)
step_size = split_variables_for_pmap(cfg['n_devices'], cfg['step_size'])

ke = pmap(create_local_kinetic_energy(vwf), in_axes=(None, 0))
pe = pmap(create_potential_energy(mol), in_axes=(0, None, None))
pmap_compute_ae_vectors_periodic_i = lambda x, y: compute_ae_vectors_periodic_i(x, y, mol.unit_cell_length)
pmap_compute_ae_vectors_periodic_i = pmap(vmap(pmap_compute_ae_vectors_periodic_i, in_axes=(0, None)), in_axes=(0, None))
pmap_compute_ee_vectors_i = pmap(vmap(compute_ee_vectors_i, in_axes=(0,)), in_axes=(0,))



System: 
 n_atoms = 2 
 n_up    = 3 
 n_down  = 3 
 n_el    = 6 

Ansatz: 
 n_layers = 2 
 n_det    = 2 
 n_sh     = 32 
 n_ph     = 8 

Cell: 
 real_basis: 
 [[6.63 0.   0.  ]
 [0.   6.63 0.  ]
 [0.   0.   6.63]] 
 reciprocal_basis: 
 [[0.94769007 0.         0.        ]
 [0.         0.94769007 0.        ]
 [0.         0.         0.94769007]] 
 real_cut         = 6.00 
 reciprocal_cut   = 6 
 kappa            = 0.50 
 volume           = 291.43 
 min_cell_width   = 6.63 
 n_periodic_input = 3 
 unit_cell_length = 6.63 

converged SCF energy = -12.1628037018806
creating wf
creating wf
sampling no infs, this could take a while
end sampling no infs


training:   0%|          | 1/5001 [00:00<?, ?it/s]

In [24]:
for step in steps:
    keys, subkeys = key_gen(keys)

    walkers, acceptance, step_size = sampler(params, walkers, subkeys, step_size)

    gparam, gwalker = pwf_grad(params, walkers)
    # gparam_nan, gwalker_nan = check_if_nan(gparam, 'x'), check_if_nan(gwalker, 'x')

    pote = pe(walkers, mol.r_atoms, mol.z_atoms)
    pote_nan = check_if_nan(pote, 'x')

    kine = ke(params, walkers)
    kine_nan = check_if_nan(kine,'x')

    ae_vectors = pmap_compute_ae_vectors_periodic_i(walkers, mol.r_atoms)
    ee_vectors = pmap_compute_ee_vectors_i(walkers)
    min_im_ee_vectors = apply_minimum_image_convention(ee_vectors, mol.unit_cell_length)
    min_im_ae_vectors = apply_minimum_image_convention(ae_vectors, mol.unit_cell_length)

    if kine_nan:
        print('nan in kinetic')
        break

    grads, e_locs = grad_fn(params, walkers)

    if cfg['opt'] == 'kfac':
        grads, state = kfac_update(step, grads, state, walkers)

    state = update(step, grads, state)
    params = get_params(state)
    # stop = capture_nan(params, 'params', stop)

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')
    steps.refresh()

    logger.log(step,
                opt_state=state,
                params=params,
                e_locs=e_locs,
                acceptance=acceptance[0])

    # if stop:
        # exit()

logger.walkers = walkers

TypeError: isnan requires ndarray or scalar arguments, got <class 'collections.OrderedDict'> at position 0.

In [ ]:
# Pretrain

params, walkers = pretrain_wf(params,
                              wf,
                              wf_orbitals,
                              mol,
                              walkers,
                              n_it=n_pre_it,
                              lr=1e-4,
                              n_eq_it=n_pre_it)

In [10]:
# vmc loop with kfac

grad_fn = create_grad_function(wf, mol)


update, get_params, kfac_update, state = kfac(kfac_wf, wf, mol, params, walkers, d0s,
                                                  lr=lr,
                                                  damping=damping,
                                                  norm_constraint=norm_constraint)


steps = trange(0, n_it, initial=0, total=n_it, desc='training', disable=None)
for step in steps:
    key, subkey = rnd.split(key)

    walkers, acceptance, step_size = sampler(params, walkers, d0s, subkey, step_size)

    grads, e_locs = grad_fn(params, walkers, d0s)

    grads, state = kfac_update(step, grads, state, walkers, d0s)

    state = update(step, grads, state)
    params = get_params(state)

    steps.set_postfix(E=f'{jnp.mean(e_locs):.6f}')
    steps.refresh()
    

print('exit')

exit


In [9]:
# periodic boundaries test

for (name, method) in zip(('LiSolid'), ('kfac')):
    config = setup(system=name,
                   pretrain=True,
                   n_pre_it=501,
                   n_walkers=2048,
                   n_layers=2,
                   n_sh=64,
                   n_ph=16,
                   opt=method,
                   n_det=8,
                   print_every=100,
                   save_every=5000,
                   lr=1e-4,
                   n_it=10000,
                   norm_constraint=1e-4,
                   damping=1e-3,
                   exp=True)

    run_vmc(**config)


Pretrain path does not exist, can't load


NameError: name 'exit' is not defined

In [3]:
config = setup(system='LiSolid',
               pretrain=True,
               n_pre_it=501,
               n_walkers=2048,
               n_layers=2,
               n_sh=64,
               n_ph=16,
               opt='kfac',
               n_det=8,
               print_every=100,
               save_every=5000,
               lr=1e-3,
               n_it=1000,
               norm_constraint=1e-4,
               damping=1e-3,
               exp=True,
               name='sampler_fix')

run_vmc(**config)

version 		 200421
seed 		 369
save_every 		 5000
print_every 		 100
exp_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4
events_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/events
models_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/models
opt_state_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/models/opt_state
pre_path 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/pretrained/s64_p16_l2_det8_1lr-4_i501.pk
timing_dir 		 /home/xmax/projects/nn_ansatz/src/experiments/LiSolid/sampler_fix/kfac_1lr-3_1d-3_1nc-4_m2048_s64_p16_l2_det8/run4/events/timing
system 		 LiSolid
r_atoms 		 [[0.5 0.5 0.5]]
z_atoms 		 [3.]
n_el 		 3
n_el_atoms 		 [3]
periodic_boundaries 		 True
cell_basis 		 [[-0

training:   0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: div got incompatible shapes for broadcasting: (2048, 3, 3), (2048, 3, 14).